In [1]:
import pandas as pd
import numpy as np
import talib as talib
from pyecharts import options as opts
from pyecharts.globals import SymbolType
from pyecharts.charts import *
# from pyecharts.

from datetime import datetime,timedelta
import matplotlib.pyplot as plt
# %matplotlib inline
#正常显示画图时出现的中文和负号
from pylab import mpl
mpl.rcParams['font.sans-serif']=['SimHei']
mpl.rcParams['axes.unicode_minus']=False

#使用tushare获取交易数据
#设置token
import tushare as ts
#注意token更换为你在tushare网站上获取的
token='36d50a0386e9066b5c79c2de58ea2d3a9b980020041adc910f76bfc7'
pro=ts.pro_api(token)
index={'上证综指': '000001.SH',
        '深证成指': '399001.SZ',
        '沪深300': '000300.SH',
        '创业板指': '399006.SZ',
        '上证50': '000016.SH',
        '中证500': '000905.SH',
        '中小板指': '399005.SZ',
        '上证180': '000010.SH'}
#获取当前交易的股票代码和名称
def get_code():
    df = pro.stock_basic(exchange='', list_status='L')
    codes=df.ts_code.values
    names=df.name.values
    stock=dict(zip(names,codes))
    #合并指数和个股成一个字典
    stocks=dict(stock,**index)
    return stocks

#获取行情数据
def get_daily_data(stock,start,end):
    #如果代码在字典index里，则取的是指数数据
    code=get_code()[stock]
    if code in index.values():
        df=pro.index_daily(ts_code=code,start_date=start, end_date=end)
    #否则取的是个股数据
    else:
        df=pro.daily(ts_code=code, adj='qfq',start_date=start, end_date=end)
    #将交易日期设置为索引值
    df.index=pd.to_datetime(df.trade_date)
    df=df.sort_index()
    #计算收益率
    df['ret']=df.close/df.close.shift(1)-1
    return df

def get_line(xdatas, data_dic):
    line= Line()
    line.add_xaxis(xaxis_data=xdatas)
    for key in data_dic:
        value = data_dic[key];
        line.add_yaxis(
            series_name=value['name'],
            y_axis=value['datas'],
            symbol="none",
            is_smooth=True,
            is_hover_animation=False,
            linestyle_opts=opts.LineStyleOpts(width=2, opacity=0.9),
            label_opts=opts.LabelOpts(is_show=False),
        )
    line.set_global_opts(xaxis_opts=opts.AxisOpts(type_="category"))
    return line

def get_kline (xdatas,kdatas,stock_name):
    c = (
        Kline()
        .add_xaxis(xaxis_data=xdatas)
        .add_yaxis(
            series_name='kline',
            y_axis=kdatas
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(
                    title=stock_name,
                    subtitle="MA",
            ),
            xaxis_opts=opts.AxisOpts(
                        is_scale=True,
                        type_="category",
                        splitarea_opts=opts.SplitAreaOpts(
                        is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
                    ),
            ),
            yaxis_opts=opts.AxisOpts(
                    is_scale=True,
                    splitarea_opts=opts.SplitAreaOpts(
                        is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
                    ),
            ),
            legend_opts=opts.LegendOpts(
                is_show=False, pos_bottom=10, pos_left="center"
            ),
            datazoom_opts=[
                opts.DataZoomOpts(
                    is_show=True,
                    xaxis_index=[0,1],
                    type_="slider",
                    pos_top="90%",
                    range_start=0,
                    range_end=100,
                ),
#                 opts.DataZoomOpts(
#                     is_show=True,
#                     yaxis_index=[0],
#                     type_="slider",
#                     orient = 'vertical',
# #                     pos_top="90%",
#                     pos_right="0%",
#                     range_start=0,
#                     range_end=100,
#                 ),
            ],
            tooltip_opts=opts.TooltipOpts(
                    trigger="axis",
                    axis_pointer_type="cross",
                    background_color="rgba(245, 245, 245, 0.8)",
                    border_width=1,
                    border_color="#ccc",
                    textstyle_opts=opts.TextStyleOpts(color="#000"),
            ),
            visualmap_opts=opts.VisualMapOpts(
                    is_show=True,
                    dimension=2,
                    series_index=9,
                    is_piecewise=True,
                    pieces=[
                        {"value": 1, "color": "#ec0000"},
                        {"value": -1, "color": "#00da3c"},
                    ],
                ),
                axispointer_opts=opts.AxisPointerOpts(
                    is_show=True,
                    link=[{"xAxisIndex": "all"}],
                    label=opts.LabelOpts(background_color="#777"),
                ),
                brush_opts=opts.BrushOpts(
                    x_axis_index="all",
                    brush_link="all",
                    out_of_brush={"colorAlpha": 0.1},
                    brush_type="lineX",
                ),
        )
    )
    return c


def get_volome(xdatas,vols,kdatas): 
    bar = (
        Bar()
        .add_xaxis(xaxis_data=xdatas)
        .add_yaxis(
            series_name="Volume",
            yaxis_data=[
                [i, vols[i], 1 if kdatas[i][0] > kdatas[i][1] else -1]
                for i in range(len(kdatas))
            ],
    #         yaxis_data = vols,
            xaxis_index=1,
            yaxis_index=1,
            label_opts=opts.LabelOpts(is_show=False)
        )
        .set_global_opts(
            xaxis_opts=opts.AxisOpts(
                type_="category",
                is_scale=True,
                grid_index=1,
                boundary_gap=False,
                axisline_opts=opts.AxisLineOpts(is_on_zero=False),
                axistick_opts=opts.AxisTickOpts(is_show=False),
                splitline_opts=opts.SplitLineOpts(is_show=False),
                axislabel_opts=opts.LabelOpts(is_show=False),
                split_number=20,
                min_="dataMin",
                max_="dataMax",
            ),
            yaxis_opts=opts.AxisOpts(
                grid_index=1,
                is_scale=True,
                split_number=2,
                axislabel_opts=opts.LabelOpts(is_show=False),
                axisline_opts=opts.AxisLineOpts(is_show=False),
                axistick_opts=opts.AxisTickOpts(is_show=False),
                splitline_opts=opts.SplitLineOpts(is_show=False),
            ),
            legend_opts=opts.LegendOpts(is_show=False),
        )
    )
    return bar

def my_strategy(data):
    x1=data.close>data.up
    x2=data.close.shift(1)<data.up.shift(1)
    x=x1&x2
    y1=data.close<data.down
    y2=data.close.shift(1)>data.down.shift(1)
    y=y1&y2
    data.loc[x,'signal']='buy'
    data.loc[y,'signal']='sell'
    buy_date=(data[data.signal=='buy'].index).strftime('%Y%m%d')
    sell_date=(data[data.signal=='sell'].index).strftime('%Y%m%d')
    buy_close=data[data.signal=='buy'].close.round(2).tolist()
    sell_close=data[data.signal=='sell'].close.round(2).tolist()
    return (buy_date,buy_close,sell_date,sell_close)

def get_grid(stock_name,xdatas,kdatas,linetec_dic,vols):
    kline = get_kline(xdatas, kdatas,stock_name)
    line = get_line(xdatas, linetec_dic)
    bar = get_volome(xdatas, vols, kdatas)
    overlap_kline_line = kline.overlap(line)
    grid_chart = Grid(init_opts=opts.InitOpts(width="900px",height="800px"))
    grid_chart.add(
        overlap_kline_line,
        grid_opts=opts.GridOpts(pos_left="6%", pos_right="8%", height="60%"),
    )
    
    grid_chart.add(
        bar,
        grid_opts=opts.GridOpts(
            pos_left="6%", pos_right="8%", pos_top="70%", height="10%"
        ),
    )
    
    return grid_chart

In [150]:
sma_periods = [8,17,25,99]
stock_name = '科信技术'
hs=get_daily_data(stock_name,'20160101','')[['trade_date','close','open','high','low','vol']]
v1=np.array(hs.loc[:,['open','close','low','high']])
trade_dates = np.array(hs.trade_date)
vols = np.array(hs['vol'])

#最近N1个交易日最高价
hs['up']=talib.MAX(hs.high,timeperiod=20).shift(1)
#最近N2个交易日最低价
hs['down']=talib.MIN(hs.low,timeperiod=10).shift(1)
#每日真实波动幅度
hs['ATR']=talib.ATR(hs.high,hs.low,hs.close,timeperiod=20)
tul_dic={}
tul_dic['upLine'] = {
    'name':'上轨道线',
    'datas': np.array(hs['up']).round(1).tolist()
}
tul_dic['downLine'] = {
    'name':'下轨道线',
    'datas': np.array(hs['down']).round(1).tolist()    
}

sma_dic= {}
for period in sma_periods:
    name = "SMA" + str(period)
    sma = talib.SMA(np.array(hs.close),period)
    sma_dic[name]={}
    sma_dic[name]['name'] = name;
    sma_dic[name]['datas'] = sma.round(2).tolist();

bd,bc,sd,sc=my_strategy(hs)

In [151]:
kline=get_kline(trade_dates.tolist(), v1.tolist(),stock_name)
# kline.render_notebook();

# lineTul = getline()

esbuy = (
    EffectScatter()
    .add_xaxis(np.array(bd).tolist())
    .add_yaxis("buy", bc)
    .set_global_opts(xaxis_opts=opts.AxisOpts(type_="category"))
#     .set_global_opts(title_opts=opts.TitleOpts(title="EffectScatter-基本示例"))
)
kline.overlap(esbuy)

esSell = (
    EffectScatter()
    .add_xaxis(np.array(sd).tolist())
    .add_yaxis(series_name="sell",y_axis=sc,symbol=SymbolType.TRIANGLE)
    .set_global_opts(xaxis_opts=opts.AxisOpts(type_="category"))
#     .set_global_opts(title_opts=opts.TitleOpts(title="EffectScatter-基本示例"))
)


kline.overlap(esSell)

line = get_line(trade_dates.tolist(), sma_dic)
overlap_kline_line = kline.overlap(line)
line = get_line(trade_dates.tolist(), tul_dic)
overlap_kline_line = kline.overlap(line)


bar = get_volome(trade_dates.tolist(), vols.tolist(), v1.tolist())

grid_chart = Grid(init_opts=opts.InitOpts(width="900px",height="800px"))
grid_chart.add(
    kline,
    grid_opts=opts.GridOpts(pos_left="6%", pos_right="8%", height="60%"),
)
    
grid_chart.add(
    bar,
    grid_opts=opts.GridOpts(
    pos_left="6%", pos_right="8%", pos_top="70%", height="10%"
    ),
)

tab = Tab()
tab.add(grid_chart,'海龟图')
    
tab.render_notebook()

In [13]:
print(hs)

           trade_date  close   open   high    low         vol     up   down  \
trade_date                                                                    
2016-01-04   20160104  11.33  12.00  12.03  11.23   563497.87    NaN    NaN   
2016-01-05   20160105  11.40  11.27  11.57  11.15   663269.95    NaN    NaN   
2016-01-06   20160106  11.53  11.42  11.56  11.39   515706.44    NaN    NaN   
2016-01-07   20160107  10.94  11.41  11.41  10.91   174761.10    NaN    NaN   
2016-01-08   20160108  11.12  11.21  11.29  10.90   747527.58    NaN    NaN   
...               ...    ...    ...    ...    ...         ...    ...    ...   
2019-09-10   20190910  14.55  14.60  14.64  14.41   796870.64  15.22  14.05   
2019-09-11   20190911  14.56  14.60  14.71  14.46   818614.21  15.22  14.05   
2019-09-12   20190912  14.68  14.66  14.78  14.53  1150687.64  15.20  14.08   
2019-09-16   20190916  14.45  14.70  14.72  14.40   929894.14  15.20  14.10   
2019-09-17   20190917  14.24  14.36  14.45  14.18   

In [149]:
hs.tail()

,trade_date,close,open,high,low,vol,up,down,ATR,signal
trade_date,,,,,,,,,,
2019-09-12,20190912,38.26,37.73,38.34,37.26,46868.26,42.20,36.50,1.750106,NaN
2019-09-16,20190916,41.05,38.60,41.99,38.58,187177.46,42.20,36.50,1.849101,NaN
2019-09-17,20190917,42.79,41.35,43.11,41.18,200593.83,42.20,36.72,1.859646,buy
2019-09-18,20190918,41.95,42.79,42.79,40.67,133139.09,43.11,36.80,1.872664,NaN
2019-09-19,20190919,44.33,42.20,44.84,41.42,159190.12,43.11,37.26,1.950031,buy


In [24]:
hs1 = hs.copy()

In [88]:
buycount = hs1[hs1['signal'] == 'buy']['close'] / hs1[hs1['signal'] == 'buy']['ATR'] / 10
buycount.name = 'sss'

hs2=pd.concat([hs1,buycount], axis=1)

x = hs1['signal']=='sell'
y = hs1['signal']=='buy'
x | y

hs2[x|y].apply(lambda x:(
    print (x['trade_date'])
),axis=1)
# hs1[hs1['signal']=='buy']['B'] = buycount.value

20160122
20160126
20160318
20160419
20160509
20160607
20160614
20160727
20160825
20160913
20160926
20161104
20161109
20161202
20161212
20161214
20170112
20170322
20170411
20170502
20170505
20170510
20170605
20170607
20170613
20170630
20170717
20170802
20170811
20170906
20170911
20170918
20170920
20170926
20171009
20171019
20171030
20171113
20171121
20171206
20171227
20180117
20180201
20180206
20180209
20180306
20180312
20180320
20180323
20180328
20180403
20180417
20180502
20180521
20180530
20180611
20180615
20180716
20180719
20180802
20180820
20180829
20180905
20180910
20180919
20180928
20181011
20181022
20181112
20181119
20181127
20181219
20181227
20190102
20190118
20190214
20190225
20190417
20190506
20190509
20190520
20190523
20190709
20190715
20190723
20190726
20190805
20190813
20190823


trade_date
2016-01-22    None
2016-01-26    None
2016-03-18    None
2016-04-19    None
2016-05-09    None
              ... 
2019-07-23    None
2019-07-26    None
2019-08-05    None
2019-08-13    None
2019-08-23    None
Length: 89, dtype: object

In [87]:
hs2.loc['2016-01-22':'2016-01-22'].index[0]

Timestamp('2016-01-22 00:00:00')

In [106]:
import datetime
import time
date=hs2['trade_date']['2016-01-22']

In [104]:
dt=datetime.datetime.strptime(date,'%Y%m%d')
dt

datetime.datetime(2016, 1, 22, 0, 0)

In [107]:
time.mktime(dt.timetuple())

1453392000.0

In [128]:
dtnxt = dt + timedelta(days=3)
dtnxt
hs2['vol','close'][0]


KeyError: ('vol', 'close')

In [139]:
hs2 = hs2.reset_index(drop=True)


In [140]:
hs2

,trade_date,close,open,high,low,vol,up,down,ATR,signal,sss
0,20160104,58.76,64.40,64.40,58.40,114880.14,NaN,NaN,NaN,NaN,NaN
1,20160105,59.35,55.80,60.61,55.80,166704.92,NaN,NaN,NaN,NaN,NaN
2,20160106,60.42,59.50,60.78,59.37,98824.86,NaN,NaN,NaN,NaN,NaN
3,20160107,55.41,59.00,59.47,55.00,41293.10,NaN,NaN,NaN,NaN,NaN
4,20160108,59.43,58.00,60.68,56.70,203567.18,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
901,20190911,51.11,51.26,51.68,50.90,116357.20,52.19,48.80,1.196311,NaN,NaN
902,20190912,50.79,51.30,51.38,50.36,99144.81,52.19,48.80,1.187495,NaN,NaN
903,20190916,51.16,51.35,51.36,50.85,83787.72,52.19,49.01,1.156620,NaN,NaN
904,20190917,49.97,51.20,51.23,49.68,105597.65,52.19,49.01,1.176289,NaN,NaN


In [142]:
hs2.set_index('trade_date',drop=False)

,trade_date,close,open,high,low,vol,up,down,ATR,signal,sss
trade_date,,,,,,,,,,,
20160104,20160104,58.76,64.40,64.40,58.40,114880.14,NaN,NaN,NaN,NaN,NaN
20160105,20160105,59.35,55.80,60.61,55.80,166704.92,NaN,NaN,NaN,NaN,NaN
20160106,20160106,60.42,59.50,60.78,59.37,98824.86,NaN,NaN,NaN,NaN,NaN
20160107,20160107,55.41,59.00,59.47,55.00,41293.10,NaN,NaN,NaN,NaN,NaN
20160108,20160108,59.43,58.00,60.68,56.70,203567.18,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
20190911,20190911,51.11,51.26,51.68,50.90,116357.20,52.19,48.80,1.196311,NaN,NaN
20190912,20190912,50.79,51.30,51.38,50.36,99144.81,52.19,48.80,1.187495,NaN,NaN
20190916,20190916,51.16,51.35,51.36,50.85,83787.72,52.19,49.01,1.156620,NaN,NaN
